## CCNSS 2018: Module 2 Tutorial 4
# Computational psychiatry assignment

*Please execute the cell below to initialize the notebook environment*

In [1]:
from IPython.display import HTML
from IPython.display import display
import matplotlib.pyplot as plt    # import matplotlib
import numpy as np                 # import numpy
import scipy as sp                 # import scipy
import  pandas  as  pd             # import pandas
import math                        # import basic math functions
import random                      # import basic random number generator functions
import time                        # import time function to time minimize
from scipy.optimize import minimize #import optimization functions
from scipy.stats import beta       # import beta distribution for BetaBinomial model
from scipy.optimize import curve_fit
import seaborn as sns

fig_w, fig_h = (6, 4)
plt.rcParams.update({'figure.figsize': (fig_w, fig_h)})
plt.style.use('ggplot')

ModuleNotFoundError: No module named 'seaborn'

In [3]:
# This code allows to call the function 'hide_toggle' that shows/hides solutions for each exercise

def hide_toggle(for_next=False):
    this_cell = """$('div.cell.code_cell.rendered.selected')"""
    next_cell = this_cell + '.next()'

    toggle_text = 'Show/hide Solution below'  # text shown on toggle link
    target_cell = this_cell  # target cell to control with toggle
    js_hide_current = ''  # bit of JS to permanently hide code in current cell (only when toggling next cell)

    if for_next:
        target_cell = next_cell
        toggle_text += ' '
        js_hide_current = this_cell + '.find("div.input").hide();'

    js_f_name = 'code_toggle_{}'.format(str(random.randint(1,2**64)))

    html = """
        <script>
            function {f_name}() {{
                {cell_selector}.find('div.input').toggle();
            }}

            {js_hide_current}
        </script>

        <a href="javascript:{f_name}()">{toggle_text}</a>
    """.format(
        f_name=js_f_name,
        cell_selector=target_cell,
        js_hide_current=js_hide_current, 
        toggle_text=toggle_text
    )

    return HTML(html)

---

## Objectives


In this notebook we'll use our previous knowledge and skills at analysing data and fitting probabilistic generative models of behaviour.
We will analyze, plot, and fit data to a synthetic dataset of Healthy controls (CTR) and Patients with Major Depressive Disorder (MDD).

- Extract and clean-up data
- Analyze and plot behavioural data
- Fit BetaBinomial model to dataset
- Plot parameters recovered for each group
- Develop modifications of the BetaBinomial model and constrained nested BetaBinomial models
- Do model comparison and model selection with different models on each dataset

---

## Background

Computational psychiatry is a young field in expansion at the intersection between computational neuroscience and psychiatry. This discipline builds on the initial efforts in the 80’s using connectionist models, but has also evolved to get closer to the physiological substrate and to more testable predictions. Although psychiatric disorders are characterised essentially by their high-level symptoms, following Marr’s principles, computational models can help formalise symptoms and hypotheses to bridge the gap between neurobiology and psychiatry. 

That is, computational models are able to provide a normative framework to explicitly define and rigorously test competing hypotheses of mental disorders, while providing a link between different levels of descriptions.

For example, modelling for psychiatry can be done using a `deductive` or `abductive` approach, each of which can lead to different predictions for psychiatry. 

Using a `deductive approach` scientists can start from the premise of known neurobiological deficits observed in mental disorders, and implement these deficits in a computational model. The performance of the model is then compared to this of patients. If the model can account for the performance observed in patients, it provides a mechanistic account to bridge biological abnormalities to behaviour or neural activity.

Using the `abductive approach` on the other hand, scientists can start from the premise of a model of normal behaviour and alter the model in multiple ways to generate distinct novel hypotheses of brain dysfunction. All these models are then fitted to the behaviour of patients to find which hypothesis (different models) accounts best for their performance and/or deficits. The winning hypothesis can then be refined in an attempt to explain the deficits at lower levels of description, or used to devise new experimental tests that will precisely assay the dysfunction suggested by the winning hypothesis.

In this tutorial we will use an `abductive approach`. That is, we will use a model of correct (healthy) behaviour to solve our task, and fit the model to the data of patients. We will then alter the model to see if it can better explain patient behaviour and/or deficits. 

Alternatively, if we find that patients and controls use the same model to solve the task, we will look at the range of parameters extracted for each group and see if patients and controls differ on some parameters.

***EXERCISE 1***

![](./figures/Optimism_task.png)

We collected a new dataset from the optimism task (as seen above and descibed in Tutorial 3).
We recruited 100 participants, 50 Healthy controls, and 50 patients with Major Depressive Disorder.

The data is contained in the data file `optimism_comp_psychiatry.csv`. The first 50 subjects are patients, the last 50 subjects are controls.

Load and clean-up the data.

***Suggestions***

* Read the data from file 'optimism_comp_psychiatry.csv'
* Each line is one decision trial, the columnns encode
    - the subject number
    - the true expected value of the fractal stimulus
    - the expected value of the target stimulus
    - the decision 
    - the reaction time
* decision / choice is encoded as: 1 = chooses fractal stimulus, 2 = chooses target stimulus
* replace numbers '1' and '2' for 'fractal' and 'target' to 1 for fractal and 0 for target
* Make sure the field 'choice' is a number
* print the first 5 rows of the data (hint: look up function 'head()' in pandas )

In [4]:
#insert your code here

hide_toggle(for_next=True)

In [ ]:
### Solution

df = pd.read_csv('data/optimism_comp_psychiatry.csv',delimiter=',')

df.replace({'choice': {1.: '1', 2.: '0'},
                 }, inplace=True)

df['choice'] = df['choice'].astype(np.float)

df.head()

***Expected Output***

![](./figures/expected_ex1.png)

***EXERCISE 2***

Calculate and plot the psychometric function for all controls (Participants with subject_id 50 to 100) 

***Suggestions***

* Loop over participants, and load their data sequentially
    * Calculate the psychometric curve for a given participant data (hint: the data has the same form as in tutorial 3, so you may reuse your previous functions)
    * Calculate the sigmoid fit to the participant's data (hint: use the sigmoid function defined in Tutorial 3, you may want to use `scipy.optimize.curve_fit`, but this time fit the sigmoid curve to the raw participant data instead of the bins for the pre-computed psychometric curve)
* Note how subjects can have different psychometric function that are shifted either left or right; What do you think this means? (hint: think of it in terms of an optimism/pessimism bias)
* Which participants appear more 'optimistic' ?
* Display 2 subplots, one with the true psychometric curve for all subjects, another with the fitted sigmoids for all subjects

In [5]:
#insert your code here

hide_toggle(for_next=True)

In [ ]:
### Solution

def sigmoid(x, x0, k):
    y = 1 / (1 + np.exp(-k*(x-x0)))
    return y

params_sig=np.full((100,2),np.nan)

fig, ax = plt.subplots(1,2,figsize=(15,7),sharey=True)

# Create column 'diff_frac2targ' which computes difference in expected value between fractal and target
df['diff_frac2targ']=df['bin_coef']-df['target_coef']

# Define the bins for plotting psychometric curve
freq_bins= np.array([-1.01, -0.5, 0, 0.5, 1.01])

#Create 'binned' column and bin each trial as belonging to either bin defined in freq_bins
df['binned']= pd.cut(df['diff_frac2targ'], bins=freq_bins)

#Select data for subject 1 only
for i in range(50,100):
    
    df_subj = df.loc[(df['subject_id']==i+1)]

    #Groupby data for subject i
    choice_groupby_binned = df_subj['choice'].groupby(df_subj['binned'])
    
    #Print sigmoid fit to the data
    my_vals=np.array(choice_groupby_binned.mean())
    my_vals=my_vals[~np.isnan(my_vals)]
    popt, pcov = curve_fit(sigmoid, df_subj['diff_frac2targ'], df_subj['choice'])
    params_sig[i]=popt;
    ax[1].plot(df_subj['diff_frac2targ'], df_subj['choice'], 'ob',  alpha=0.01, label='data')
    ax[1].plot(np.linspace(-1, 1, 100),sigmoid(np.linspace(-1, 1, 100), *popt),'-b',lw=2,alpha=0.1)
    ax[0].plot(range(len(my_vals)), my_vals, '-b', lw=2, alpha=0.1, label='data')
    ax[0].set_xticks(range(len(my_vals)))
    ax[0].set_xticklabels(['[-1,-0.5]' ,'[-0.5,0]' ,'[0,0.5]' ,'[0.5,1]'])
    
ax[0].set_ylabel('Probability of choosing `fractal`')
ax[0].set_xlabel('Binned : Difference in reward probability (Fractal-Target)')
ax[0].set_title('Psychometric function for: Controls')
ax[1].set_ylabel('Probability of choosing `fractal`')
ax[1].set_xlabel('True Difference in reward probability (Fractal-Target)')
ax[1].set_title('Fitted Psychometric function for: Controls')


***Expected Output***

![](./figures/expected_ex2.png)

***EXERCISE 3***

Calculate and plot the psychometric function for all patients & controls on the same plot

***Suggestions***

* Loop over participants, and load their data sequentially
    * Use your previous function which calculates the psychometric curve for a given participant data.
    * Plot the participant fitted sigmoidal response with an alpha=0.05
* Note how subjects can have different psychometric function that are shifted either left/right or up/down; What do you think this means? (hint: think of it in terms of a bias)

In [6]:
#insert your code here

hide_toggle(for_next=True)

In [ ]:
### Solution

params_sig=np.full((100,2),np.nan)
psychometric_values=np.full((100,4),np.nan)

fig, ax = plt.subplots(1,2,figsize=(15,7),sharey=True)

# Create column 'diff_frac2targ' which computes difference in expected value between fractal and target
df['diff_frac2targ']=df['bin_coef']-df['target_coef']

# Define the bins for plotting psychometric curve
freq_bins= np.array([-1.01, -0.5, 0, 0.5, 1.01])

#Create 'binned' column and bin each trial as belonging to either bin defined in freq_bins
df['binned']= pd.cut(df['diff_frac2targ'], bins=freq_bins)

#Select data for subject 1 only
for i in range(100):
    
    df_subj = df.loc[(df['subject_id']==i+1)]

    #Groupby data for subject i
    choice_groupby_binned = df_subj['choice'].groupby(df_subj['binned'])
    
    #Print sigmoid fit to the data
    my_vals=np.array(choice_groupby_binned.mean())
    psychometric_values[i,:]=my_vals;
    my_vals=my_vals[~np.isnan(my_vals)]
    popt, pcov = curve_fit(sigmoid, df_subj['diff_frac2targ'], df_subj['choice'])
    params_sig[i]=popt;
    if i < 50:
        ax[0].plot(range(len(my_vals)), my_vals, '-r', lw=2, alpha=0.1, label='Patients')
        ax[1].plot(df_subj['diff_frac2targ'], df_subj['choice'], 'or',  alpha=0.01, label='data')
        ax[1].plot(np.linspace(-1, 1, 100),sigmoid(np.linspace(-1, 1, 100), *popt),'-r',lw=2,alpha=0.1)
    else:
        ax[0].plot(range(len(my_vals)), my_vals, '-b', lw=2, alpha=0.1, label='Controls')
        ax[1].plot(df_subj['diff_frac2targ'], df_subj['choice'], 'ob',  alpha=0.01, label='data')
        ax[1].plot(np.linspace(-1, 1, 100),sigmoid(np.linspace(-1, 1, 100), *popt),'-b',lw=2,alpha=0.1)
    ax[0].set_xticks(range(len(my_vals)))
    ax[0].set_xticklabels(['[-1,-0.5]' ,'[-0.5,0]' ,'[0,0.5]' ,'[0.5,1]'])
    
ax[0].set_ylabel('Probability of choosing `fractal`')
ax[0].set_xlabel('Binned : Difference in reward probability (Fractal-Target)')
ax[0].set_title('Psychometric function for: \n Controls(Blue) and Patients (Red)')
ax[1].set_ylabel('Probability of choosing `fractal`')
ax[1].set_xlabel('True Difference in reward probability (Fractal-Target)')
ax[1].set_title('Fitted Psychometric function for: \n Controls(Blue) and Patients (Red)')

***Expected Output***

![](./figures/expected_ex3.png)

***EXERCISE 4***

We have plotted estimated sigmoidal fit and raw behaviour of participants, and it looks as if there may be some some shift of the psychometric curve between Patients (red) and Controls (blue).

Let's try and quantify this by summarizing the data per groups.

***Suggestions***

* Plot the mean and standard deviation of the psychometric curve at each bins, for each group using errorbars.
* The mean+- standard deviation may hide valuable information (i.e. non-normally distributed data, skewness, kurtosis, etc). 
Let's reproduce the same graph but this time using violin plots stacked onto one another. This will give us information regarding the shape and skewness in the data (violin plot). (hint: you may want to use the function  `violinplot` from `matplotlib`. Alternatively you may use `seaborn`)
* Plot the smoothed histogram (also known as Kernel Density Estimation -- KDE) of the parameters extracted from the sigmoid fits to the data. Plot one KDE per group and parameter. (hint: you may want to use the function `kdeplot` from `seaborn`)
    * Considering the amount of overlap between the two groups over the parameter space, do you believe that they are indeed two different populations? 
    * What do you think it means, if one parameter is different but not the other?
    * (optional) Run statistics on your parameters and report them (hint: you may want to use independent t-tests, assuming the data is normally distributed)
* (optional) Run a two-way repeated-measure ANOVA on the psychometric curves (Report the Group, Bin, and Group x Bin interaction)


In [7]:
#insert your code here

hide_toggle(for_next=True)

In [ ]:
### Solution
fig, ax = plt.subplots(2,2,figsize=(20,12))

ax[0,0].errorbar(range(len(my_vals)), np.mean(psychometric_values[0:49,:],axis=0), yerr=np.std(psychometric_values[0:49,:],axis=0),marker='o',lw=2,alpha=0.6,label='Patients')
ax[0,0].errorbar(range(len(my_vals)), np.mean(psychometric_values[50:100,:],axis=0), yerr=np.std(psychometric_values[50:100,:],axis=0),marker='o',lw=2,alpha=0.6,label='Controls')
ax[0,0].legend(loc='best', frameon=False)
ax[0,0].set_xticks(range(len(my_vals)))
ax[0,0].set_xticklabels(['[-1,-0.5]' ,'[-0.5,0]' ,'[0,0.5]' ,'[0.5,1]'])
ax[0,0].set_ylabel('Probability of choosing `fractal`')
ax[0,0].set_xlabel('Binned : Difference in reward probability (Fractal-Target)')
ax[0,0].set_title('Mean Psychometric function for: \n Controls(Blue) and Patients (Red)')

ax[0,1].violinplot(psychometric_values[0:49,:],showmeans=False,showmedians=True,showextrema=False)
ax[0,1].plot(np.array([1, 2, 3, 4]),np.mean(psychometric_values[0:49,:],axis=0),'--r',marker='o',lw=2,alpha=0.3,label='Patients')
ax[0,1].violinplot(psychometric_values[50:100,:],showmeans=False,showmedians=True,showextrema=False)
ax[0,1].plot(np.array([1, 2, 3, 4]),np.mean(psychometric_values[50:100,:],axis=0),'.-b',marker='o',lw=2,alpha=0.3,label='Controls')
ax[0,1].legend(loc='best', frameon=False)
ax[0,1].set_xticks(np.array([1, 2, 3, 4]))
ax[0,1].set_xticklabels(['[-1,-0.5]' ,'[-0.5,0]' ,'[0,0.5]' ,'[0.5,1]'])
ax[0,1].set_ylabel('Probability of choosing `fractal`')
ax[0,1].set_xlabel('Binned : Difference in reward probability (Fractal-Target)')
ax[0,1].set_title('Mean, Mode & distribution of Psychometric function for: \n Controls(Blue) and Patients (Red)')

params_sig0=params_sig[params_sig[:,0]<0.75,0]; #remove incorrectly estimated parameters
params_sig1=params_sig[params_sig[:,1]<10,1];   #remove incorrectly estimated parameters

sns.kdeplot(params_sig0[0:49],shade=True, color='red', ax=ax[1,0], label='KDE Patient')
sns.kdeplot(params_sig0[50:99],shade=True, color='blue', ax=ax[1,0], label='KDE Control')
ax[1,0].set_ylabel('PDF estimated from data')
ax[1,0].set_xlabel('`x0`: Sigmoid bias (mid-point)')
ax[1,0].set_title('Kernel Density Estimation of Sigmoid Parameter `x0`')

#ax[1,0].set_xlim([-0.75, 0.75])
sns.kdeplot(params_sig1[0:49],shade=True, color='red', ax=ax[1,1], label='KDE Patient')
sns.kdeplot(params_sig1[50:99],shade=True, color='blue', ax=ax[1,1], label='KDE Control')
ax[1,1].set_ylabel('PDF estimated from data')
ax[1,1].set_xlabel('`k`: Sigmoid steepness (curvature)')
ax[1,1].set_title('Kernel Density Estimation of Sigmoid Parameter `k`')


***Expected Output***

![](./figures/expected_ex4.png)

***EXERCISE 5***

Depressive Realism or Healthy optimism?

Patients with Major Depression often tend to have lower scores on optimism self-reports than healthy controls. 
We could think of this as meaning that they would be more 'pessimistic' than healthy controls. 
However, it turns out that, when you ask MDD patients and controls to report the probability of something good or bad happening to them, patients tend to report accurate probabilities more often, while healthy controls overestimate the probability of something good happening to them (say: winning the lottery) and underestimate the probability of something bad happening to them (e.g.: getting robbed , or getting cancer). 
This is sometimes known as the 'optimism bias' in healthy controls, and the 'depressive realism' in patients.

In our experiment, we also asked participants to fill in a questionnaire measuring their optimism levels (found in data file: optimism_questionnaire.csv).

Lower scores in the scale suggests that a participants are more pessimistic, while high scores reflects optimism.

***Suggestions***

* Load the questionnaire scores
* Plot a Kernel density overlayed onto the histogram of questionnaire scores for each group (Patients in red, Controls in blue). hint: you may want to use the function `distplot` from `seaborn`
* What do you observe?
* Can you see a significant difference between the two groups in terms of optimism/pessimism score?
    * Does this conform to what we would expect, considering what we know of optimism/pessimism in patients and controls?


In [8]:
#insert your code here

hide_toggle(for_next=True)

In [ ]:
### Solution

df_optim = pd.read_csv('data/optimism_questionnaire.csv',delimiter=',')

df_optim['optimism_score'] = df_optim['optimism_score'].astype(np.int)
optimism = np.array(df_optim['optimism_score'])

ax=sns.distplot(optimism[0:49], bins=10,color='red', label='KDE Patient')
ax=sns.distplot(optimism[50:99], bins=10,color='blue', label='KDE Control')
ax.legend(loc='best', frameon=False)
ax.set_ylabel('Probability Density')
ax.set_xlabel('Optimism questionnaire score')

***Expected Output***

![](./figures/expected_ex5.png)

# Application: Fitting the BetaBinomial model to the data

***EXERCISE 6***

Now that we have plotted the raw behavioural data for both groups. We can try to fit the BetaBinomial model from last class to our new dataset.
    
***Suggestions***

* Import your own `Beta Binomial` model based on the work from the last tutorial (copy the functions `get_softmax`, `get_mean_BetaBinomial`, `get_negll_mean_BetaBinom`)
* Write a wrapper for the function `get_negll_mean_BetaBinom` so that you have all parameters in an array, and can load different datasets
* Load each subject sequentially and fit the model to each subjects 
    * Use the following: `initial_guess=[1,1,0.1]
    * Use the following bounds: `bounds=[(0.01,6),(0.01,6),(0.01,0.8)]`
    * Store the parameters $a$, $b$, $tau$, and the negative log_likelihoods for each subject
    * Print the parameters $a$, $b$, $tau$, `log_likelihoods` and time taken to converge for subject 1 to 5

In [9]:
#insert your code here

hide_toggle(for_next=True)

In [ ]:
### Solution

def get_softmax(x,tau):
    """
    Function that implements the softmax link function
    ----------
    x: array of 2 values 
        Success rates Ci for each option 1 & 2 (in our case success probability of Fractal vs. Target)
    tau: integer > 0 
        Temperature parameter
    
    Returns
    -------
    likelihood: 
        probability of chosing each value of x respectively, as a function of tau, and values of x
    """
    
    return (np.exp( (x - np.max(x)) / tau) / np.sum(np.exp( (x - np.max(x)) / tau), axis=0))

def get_mean_BetaBinomial(a, b, n, N):
    """
    Function that implements the BetaBinomial conjugate model and returns the mean of the posterior
    
    Parameters
    ----------
    N: integer > 0
        Number of presentations for fractal
    n: integer > 0 & =< N
        Number of rewarded presentations of the fractal
    c: float > [0..1]
    
    Returns
    -------
    negative log-likelihood: 
        probability of occurence given the parameters
    
    """
    c = ( n + a ) / (N + a + b)
    
    return c

def get_negll_mean_BetaBinom(a,b,tau, data):
    '''
    Determines the negative loglikelihood of the BetaBinomial model with softmax link function
    
    Parameters
    ----------
    parameter : array_like of float
        length 3: 1st entry is a (shape of Beta prior), 2nd is b 
        (2nd shape parameter of Beta prior), 3rd is tau (temperature parameter)
        Note: we pack mu and B in one parameter because we want to
        make it compatible for later use with sp.optimize.minimize
    data : array_like of decision trials.
        contains n, N, target_c, and choice
        
    Returns
    -------
    nll : float
        negative log-likelihood
    '''
    
    l=list()
    
    for i_trial in range(len(data)):
        n=data[i_trial,1]
        N=data[i_trial,2]
        target=data[i_trial,4]
        c_hat= get_mean_BetaBinomial(a, b, n, N)
        choice_likelihoods=get_softmax([target,c_hat],tau)+1e-100     #target always 1st elem (0), and fractal 2nd elem (1), so that it matches df['choice']=0 or 1 for target/fractal respectively  
        l.append(choice_likelihoods[int(data[i_trial,5])])        
        
    return -np.sum(np.log(l))

get_nll_mean_BetaBinomial_wrapper = lambda parameters, data: get_negll_mean_BetaBinom(parameters[0], parameters[1], parameters[2], data)

hide_toggle(for_next=True)

In [ ]:
### Solution

# Store parameters found
est_a, est_b, est_tau, ll_mean= list(), list(), list(), list()

# Optimization (find parameters that minimize nll)
initial_guess = [1,1,0.1]
bounds = [(0.01,6),(0.01,6),(0.01,0.8)] # Optimization bounds

#Select data 1 subject at a time
for i_subj in range(len(df['subject_id'].unique())):
    
    df_subj = df.loc[(df['subject_id']==i_subj+1)]
    np_subj = np.array(df_subj)
    
    t_start_optim = time.time()
    res = minimize(get_nll_mean_BetaBinomial_wrapper, 
               args=(np_subj),
               method='SLSQP',x0=initial_guess, bounds=bounds)
    ll_mean.append(-res.fun)
    t_end_optim = time.time()
    est_a.append(res.x[0])
    est_b.append(res.x[1])
    est_tau.append(res.x[2])
    if i_subj<5:
        print('Participant :'+str(i_subj+1))
        print('Estimated a:'+str(round(est_a[i_subj],2))+' , b:'+str(round(est_b[i_subj],2))+' , tau:'+str(round(est_tau[i_subj],2)) )
        print('LL :'+str(round(ll_mean[i_subj],5))+' , time taken:'+str(round(t_end_optim-t_start_optim,2))+' seconds')
        print(' ')

***Expected Output***

![](./figures/expected_ex6.png)

***EXERCISE 7***

Plot the parameters recovered in each group using violin plots and box plots

***Suggestions***

* Using distribution plots `distplot`, display the parameters recovered, per group for:
    * $a$
    * $b$
    * Mean of the prior. (hint: the mean of a Beta distribution is $\left(\frac{a}{a + b}\right)$)
    * $tau$
* Compare the mean of the prior to the self-report measures of optimism. What do you find? What do you think this means?

In [10]:
#insert your code here

hide_toggle(for_next=True)

In [ ]:
### Solution

from scipy.stats import linregress

fig,ax = plt.subplots(2,2,figsize=(15,7))

sns.distplot(est_a[0:49], bins=10,color='red', label='KDE Patient', ax=ax[0,0])
sns.distplot(est_a[50:99], bins=10,color='blue', label='KDE Control', ax=ax[0,0])
ax[0,0].legend(loc='best', frameon=False)
ax[0,0].set_ylabel('Probability Density')
ax[0,0].set_xlabel('Beta prior, parameter: `a`')

sns.distplot(est_b[0:49], bins=10,color='red', label='KDE Patient', ax=ax[0,1])
sns.distplot(est_b[50:99], bins=10,color='blue', label='KDE Control', ax=ax[0,1])
ax[0,1].legend(loc='best', frameon=False)
ax[0,1].set_ylabel('Probability Density')
ax[0,1].set_xlabel('Beta prior, parameter: `b`')

prior_mean_patients=(np.array(est_a[0:49]))/(np.array(est_a[0:49])+np.array(est_b[0:49]))
prior_mean_controls=(np.array(est_a[50:99]))/(np.array(est_a[50:99])+np.array(est_b[50:99]))

sns.distplot(prior_mean_patients, bins=10,color='red', label='KDE Patient', ax=ax[1,0])
sns.distplot(prior_mean_controls, bins=10,color='blue', label='KDE Control', ax=ax[1,0])
ax[1,0].legend(loc='best', frameon=False)
ax[1,0].set_ylabel('Probability Density')
ax[1,0].set_xlabel('Beta prior, mean: `a/(a+b)`')

sns.distplot(est_tau[0:49], bins=10,color='red', label='KDE Patient', ax=ax[1,1])
sns.distplot(est_tau[50:99], bins=10,color='blue', label='KDE Control', ax=ax[1,1])
ax[1,1].legend(loc='best', frameon=False)
ax[1,1].set_ylabel('Probability Density')
ax[1,1].set_xlabel('Temperature parameter: `tau`')

plt.figure()
plt.plot(optimism[0:49],prior_mean_patients,'or',alpha=0.4)
plt.plot(optimism[50:99],prior_mean_controls,'ob',alpha=0.4)
plt.plot(np.linspace(5,25,100), 0.045*np.linspace(5,25,100)-0.15, '--k')
plt.ylabel('Mean of Prior: `a/(a+b)`');
plt.xlabel('Optimism Score (Higher = Optimistic)');
plt.legend(loc='best', frameon=False)


***Expected Output***

![](./figures/expected_ex7.png)

***EXERCISE 8***

Let's try the `abductive approach` and modify the correct model of behaviour (i.e. the mean_BetaBinomial model).
We'll then be able to test these model against behaviour to find out whether our participants appear to be using the strategy (i.e. 'model') we think they're actually using.

***Suggestions***

* Alter the BetaBinomial model so that instead of taking the mean of the posterior for our behavioural outcomes, we assume participant chose the mode of the posterior. 
The mode of the posterior is defined by : $\left(\frac{n_i + \alpha - 1}{N_i + \alpha + \beta - 2}\right)$
* Alter the BetaBinomial model so that instead of taking the mean of the posterior for our behavioural outcomes, we assume participant chose the median of the posterior. 
The mode of the posterior is defined by : $\left(\frac{n_i + \alpha - 1/3}{N_i + \alpha + \beta - 1/3}\right)$
* Using your functions, calculate and print the likelihood for subject 1, with parameters $a=5$, $b=2$, $tau=0.1$ on:
    * median_Beta_Binomial
    * mode_Beta_Binomial

In [11]:
#insert your code here

hide_toggle(for_next=True)

In [ ]:
### Solution

def get_mode_BetaBinomial(a, b, n, N):
    """
    Function that implements the BetaBinomial conjugate model and returns the mean of the posterior
    
    Parameters
    ----------
    N: integer > 0
        Number of presentations for fractal
    n: integer > 0 & =< N
        Number of rewarded presentations of the fractal
    c: float > [0..1]
    
    Returns
    -------
    negative log-likelihood: 
        probability of occurence given the parameters
    
    """
    c = ( n + a -1) / (N + a + b -2)
    
    return c

def get_median_BetaBinomial(a, b, n, N):
    """
    Function that implements the BetaBinomial conjugate model and returns the mean of the posterior
    
    Parameters
    ----------
    N: integer > 0
        Number of presentations for fractal
    n: integer > 0 & =< N
        Number of rewarded presentations of the fractal
    c: float > [0..1]
    
    Returns
    -------
    negative log-likelihood: 
        probability of occurence given the parameters
    
    """
    c = ( n + a -(1/3)) / (N + a + b -(1/3))
    
    return c

def get_negll_mode_BetaBinom(a,b,tau, data):
    '''
    Determines the negative loglikelihood of the BetaBinomial model with softmax link function
    
    Parameters
    ----------
    parameter : array_like of float
        length 3: 1st entry is a (shape of Beta prior), 2nd is b 
        (2nd shape parameter of Beta prior), 3rd is tau (temperature parameter)
        Note: we pack mu and B in one parameter because we want to
        make it compatible for later use with sp.optimize.minimize
    data : array_like of decision trials.
        contains n, N, target_c, and choice
        
    Returns
    -------
    nll : float
        negative log-likelihood
    '''
    
    l=list()
    
    for i_trial in range(len(data)):
        n=data[i_trial,1]
        N=data[i_trial,2]
        target=data[i_trial,4]
        c_hat= get_mode_BetaBinomial(a, b, n, N)
        choice_likelihoods=get_softmax([target,c_hat],tau)+1e-100     #target always 1st elem (0), and fractal 2nd elem (1), so that it matches df['choice']=0 or 1 for target/fractal respectively  
        l.append(choice_likelihoods[int(data[i_trial,5])])        
        
    return -np.sum(np.log(l))

def get_negll_median_BetaBinom(a,b,tau, data):
    '''
    Determines the negative loglikelihood of the BetaBinomial model with softmax link function
    
    Parameters
    ----------
    parameter : array_like of float
        length 3: 1st entry is a (shape of Beta prior), 2nd is b 
        (2nd shape parameter of Beta prior), 3rd is tau (temperature parameter)
        Note: we pack mu and B in one parameter because we want to
        make it compatible for later use with sp.optimize.minimize
    data : array_like of decision trials.
        contains n, N, target_c, and choice
        
    Returns
    -------
    nll : float
        negative log-likelihood
    '''
    
    l=list()
    
    for i_trial in range(len(data)):
        n=data[i_trial,1]
        N=data[i_trial,2]
        target=data[i_trial,4]
        c_hat= get_median_BetaBinomial(a, b, n, N)
        choice_likelihoods=get_softmax([target,c_hat],tau)+1e-100     #target always 1st elem (0), and fractal 2nd elem (1), so that it matches df['choice']=0 or 1 for target/fractal respectively  
        l.append(choice_likelihoods[int(data[i_trial,5])])        
        
    return -np.sum(np.log(l))

hide_toggle(for_next=True)

In [12]:
#insert your code here

hide_toggle(for_next=True)

In [ ]:
### Solution

df_subj = df.loc[(df['subject_id']==i_subj+1)]
np_subj = np.array(df_subj)

print('Participant :'+str(1)+' , using a: 5 , b: 2 , tau: 0.1' )
print('LL model median_BetaBinom: '+str(-get_negll_median_BetaBinom(5,2,0.1, np_subj)))
print('LL model mode_BetaBinom: '+str(-get_negll_mode_BetaBinom(5,2,0.1, np_subj)))


***Expected Output***

![](./figures/expected_ex8.png)

***EXERCISE 9***

Fit the 3 different models of behaviour to the data of controls and patients, and use boundaries to create constrained nested models.

Do model comparison using the BIC on each group separately, and report which model wins overall and for each group.

You may want to use the following table to compare the evidence for each model and decide which model 'wins':

![](./figures/KassRaftery_BayesFactor.png)

***Suggestions***

* Fit a Null model using the BetaBinomial_mean to the controls and patient dataset (hint: you may want to restrict the range of the parameter values using the bounds of the minimization function)
* Fit a No_Bias (no prior) model using the BetaBinomial_mean to the controls and patient dataset (hint: instead of changing the model, you may want to restrict the range of the prior parameter values, so that the prior is completely uniform and un-informative)
* Fit the model BetaBinomial_mode to the controls and patient dataset
* Fit the model BetaBinomial_median to the controls and patient dataset
* Calculate the BIC per model, per subject
* Report and plot the BIC for each model, for all subjects combined, then per group. 
    * What does this suggest?

In [13]:
#insert your code here

hide_toggle(for_next=True)

In [ ]:
### Solution

get_nll_mode_BetaBinomial_wrapper = lambda parameters, data: get_negll_mode_BetaBinom(parameters[0], parameters[1], parameters[2], data)
get_nll_median_BetaBinomial_wrapper = lambda parameters, data: get_negll_median_BetaBinom(parameters[0], parameters[1], parameters[2], data)

# Store parameters found
ll_mode, ll_median , ll_null, ll_nobias= list(), list(), list(), list()
bic = np.full((100,5),np.nan)
N_trials = 60

# Optimization (find parameters that minimize nll)
initial_guess = [1,1,0.1]
bounds = [(0.01,6),(0.01,6),(0.01,0.8)] # Optimization bounds

#Select data 1 subject at a time
for i_subj in range(len(df['subject_id'].unique())):
    
    df_subj = df.loc[(df['subject_id']==i_subj+1)]
    np_subj = np.array(df_subj)
       
    t_start_optim = time.time()
    res = minimize(get_nll_mean_BetaBinomial_wrapper, 
               args=(np_subj),
               method='SLSQP',x0=[1,1,100], bounds=[(1,1),(1,1),(100,100)])
    ll_null.append(-res.fun)
    bic[i_subj,0]=-2*(-res.fun)+0*np.log(N_trials)
    
    t_start_optim = time.time()
    res = minimize(get_nll_mean_BetaBinomial_wrapper, 
               args=(np_subj),
               method='SLSQP',x0=[1,1,0.15], bounds=[(1,1),(1,1),(0.01,0.8)])
    ll_nobias.append(-res.fun)
    bic[i_subj,1]=-2*(-res.fun)+1*np.log(N_trials)
        
    t_start_optim = time.time()
    res = minimize(get_nll_mean_BetaBinomial_wrapper, 
               args=(np_subj),
               method='SLSQP',x0=initial_guess, bounds=bounds)
    ll_mean.append(-res.fun)
    bic[i_subj,2]=-2*(-res.fun)+3*np.log(N_trials)
    
    t_start_optim = time.time()
    res = minimize(get_nll_mode_BetaBinomial_wrapper, 
               args=(np_subj),
               method='SLSQP',x0=initial_guess, bounds=bounds)
    ll_mode.append(-res.fun)
    bic[i_subj,3]=-2*(-res.fun)+3*np.log(N_trials)
    
    res = minimize(get_nll_median_BetaBinomial_wrapper, 
               args=(np_subj),
               method='SLSQP',x0=initial_guess, bounds=bounds)
    ll_median.append(-res.fun)
    bic[i_subj,4]=-2*(-res.fun)+3*np.log(N_trials)

In [14]:
#insert your code here

hide_toggle(for_next=True)

In [ ]:
### Solution

fig,ax = plt.subplots(1,3,figsize=(20,7),sharey=True)

ax[0].bar(range(5),np.sum(bic,axis=0)-np.min(np.sum(bic,axis=0)),color='black')
ax[0].set_xticks(range(5))
ax[0].set_xticklabels(('Null', 'No Bias', 'Beta Mean', 'Beta Mode', 'Beta Median'))
ax[0].set_ylabel('BIC difference vs. winning model')
ax[0].set_title('Model comparison accross all subjects')
ax[0].set_yscale('log')

ax[1].bar(range(5),np.sum(bic[50:100,:],axis=0)-np.min(np.sum(bic[50:100,:],axis=0)),color='blue')
ax[1].set_xticks(range(5))
ax[1].set_xticklabels(('Null', 'No Bias', 'Beta Mean', 'Beta Mode', 'Beta Median', 'Null', 'No Bias'))
ax[1].set_ylabel('BIC difference vs. winning model')
ax[1].set_title('Model comparison for Controls')
ax[1].set_yscale('log')

ax[2].bar(range(5),np.sum(bic[0:49,:],axis=0)-np.min(np.sum(bic[0:49,:],axis=0)),color='red')
ax[2].set_xticks(range(5))
ax[2].set_xticklabels(('Null', 'No Bias', 'Beta Mean', 'Beta Mode', 'Beta Median', 'Null', 'No Bias'))
ax[2].set_ylabel('BIC difference vs. winning model')
ax[2].set_title('Model comparison for Patients')
ax[2].set_yscale('log')

***Expected Output***

![](./figures/expected_ex9.png)

***EXERCISE 10***

Since this is a synthetic dataset, we can look at our recovery of parameters.

The true prior mean, and temperature parameter for each subject is in the dataFrame column 8 and 9 respectively.

***Suggestions***

* Plot the estimated mean of the prior vs. the true mean of the prior he true prior mean.
* Plot the estimated temperature parameter against its true value
    * Does the recovery of parameters look acceptable?

In [15]:
#insert your code here

hide_toggle(for_next=True)

In [ ]:
### Solution

fig,ax = plt.subplots(1,2,figsize=(15,7))

#Select data 1 subject at a time
for i_subj in range(len(df['subject_id'].unique())):
    
    df_subj = df.loc[(df['subject_id']==i_subj+1)]
    np_subj = np.array(df_subj)
    if i_subj < 50:
        ax[0].plot((np_subj[0,6]/(np_subj[0,6]+np_subj[0,7])),(est_a[i_subj]/(est_a[i_subj]+est_b[i_subj])),'or',alpha=0.4)
        ax[1].plot(np_subj[0,9],est_tau[i_subj],'or',alpha=0.4)
    else:
        ax[0].plot((np_subj[0,6]/(np_subj[0,6]+np_subj[0,7])),(est_a[i_subj]/(est_a[i_subj]+est_b[i_subj])),'ob',alpha=0.4)
        ax[1].plot(np_subj[0,9],est_tau[i_subj],'ob',alpha=0.4)

ax[0].set_ylim([-0.05,1.05])
ax[0].set_xlim([-0.05,1.05])
ax[1].set_ylim([-0.05,1.05])
ax[1].set_xlim([-0.05,1.05])
ax[0].plot(np.arange(0.1,0.95,0.1), 1*np.arange(0.1,0.95,0.1),'--k',lw=2,label='Ideal recovery X=Y')
ax[0].set_ylabel('Estimated mean of prior');
ax[0].set_xlabel('True mean of prior');
ax[1].plot(np.arange(0.1,0.95,0.1), 1*np.arange(0.1,0.95,0.1),'--k',lw=2,label='Ideal recovery X=Y')
ax[1].set_ylabel('Estimated Temperature');
ax[1].set_xlabel('True Temperature');
ax[0].legend(loc='best', frameon=False)
ax[1].legend(loc='best', frameon=False)

***Expected Output***

![](./figures/expected_ex10.png)